# **CSE655 Homework1**

### *Part 1: Training a deep feed forward network for multidimensional regression.*
Assume that you are given a polynomial with multiple (8) inputs and multiple outputs (6) of the form:
\begin{align*}
(y_1​, y_2​, …, y_6​) = P(x_1​, x_2​, …, x_8​)
\end{align*}

\begin{align*}
y_1 &= x_1 x_3 + 1.2 x_1 x_5 - x_6 x_7 x_8 - 2 x_1^2 x_8 + x_5 \\
y_2 &= x_1 x_5 x_6 - x_3 x_4 - 3 x_2 x_3 + 2 x_2^2 x_4 - 2 x_7 x_8 - 1 \\
y_3 &= 2 x_3^2 - x_5 x_7 - 3 x_1 x_4 x_6 - x_1^2 x_2 x_4 - 1 \\
y_4 &= -x_6^3 + 2 x_1 x_3 x_8 - x_1 x_4 x_7 - 2 x_5^2 x_2 x_4 - x_8 \\
y_5 &= x_1^2 x_5 - 3 x_3 x_4 x_8 + x_1 x_2 x_4 - 3 x_6 - x_1^2 x_7 + 2 \\
y_6 &= x_1^2 x_3 x_6 - x_3 x_5 x_7 + x_3 x_4 + 2.2 x_4 + x^2_2 x_3 - 2.1
\end{align*}



In [2]:
def function(x):
    x1, x2, x3, x4, x5, x6, x7, x8 = x
    
    y1 = x1 * x3 + 1.2 * x1 * x5 - x6 * x7 * x8 - 2 * x1**2 * x8 + x5
    y2 = x1 * x5 * x6 - x3 * x4 - 3 * x2 * x3 + 2 * x2**2 * x4 - 2 * x7 * x8 - 1
    y3 = 2 * x3**2 - x5 * x7 - 3 * x1 * x4 * x6 - x1**2 * x2 * x4 - 1
    y4 = -x6**3 + 2 * x1 * x3 * x8 - x1 * x4 * x7 - 2 * x5**2 * x2 * x4 - x8
    y5 = x1**2 * x5 - 3 * x3 * x4 * x8 + x1 * x2 * x4 - 3 * x6 - x1**2 * x7 + 2
    y6 = x1**2 * x3 * x6 - x3 * x5 * x7 + x3 * x4 + 2.2 * x4 + x2**2 * x3 - 2.1
    
    return [y1, y2, y3, y4, y5, y6]

Training and validation data:
* Given the polynomials, generate 𝑁t instances of data pairs of the form: 
\begin{align*} {(x_1^t, x_2^t, x_3^t ... x_8^t), (y_1^t, y_2^t ... y_6^t)} \end{align*}
* When needed, add some noise to the training data from a normal distribution with mean 𝜇 and standard deviation 𝜎 such that the training data becomes:
\begin{align*} {(x_1^t, x_2^t, x_3^t ... x_8^t), (y_1^t + 𝑁(𝜇,𝜎), y_2^t + 𝑁(𝜇,𝜎) ... y_6^t + 𝑁(𝜇,𝜎))} \end{align*}
* Similarly, create an additional 𝑁v instances:
\begin{align*} {(x_1^v, x_2^v, x_3^v ... x_8^v), (y_1^v, y_2^v ... y_6^v)} \end{align*}
for validation of the trained model.

In [ ]:
import numpy as np

Nt = 1000 # Training Data Count
Nv = 1000 # Validation Data Count

noiseMean = 0.0
noiseStddev = 0.001

def dataGenerator(func, count):
    data = []
    loop = 0
    while loop < count:
        x = np.random.random(8).tolist()
        y = func(x)
        data.append([x, y]) 
        loop += 1

    return data

def noiseGenerator(array, noiseMean, noiseSigma):
    pass


IndentationError: expected an indented block after function definition on line 20 (187341525.py, line 23)